## Kevin Goodman-Rendall - m20210701 Data Visualization Project Spring 2022

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sat Mar 26 17:27:01 2022

@author: 'Kevin Goodman-Rendall'
"""
### DV Final Project Visualization

# Idea is to use dash with dcc.Graph animate = True to create moving representation
# on a map such as an explorer moving through a geographical location.


# Import Libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import datetime as dt
import plotly
from dash import Dash, dcc, html, Input, Output, no_update

# Import Data 
dataset = pd.read_csv('Magellin-route.txt')
timechange = pd.read_csv('timedelta.txt')

# Process Data
Datetime_series = pd.Series(name = 'Datetime', data = None, 
                            index = dataset.index, dtype = object)

for idx, date in enumerate(dataset['Date']):
    if date == 'no date':
        continue
    Datetime_series.iloc[idx] = dt.datetime.strptime(date, '%B %d %Y')

dataset = dataset.assign(Datetime=Datetime_series.values)

dataset['timedelta']=timechange

dataset['markersize']=[1]*len(dataset)

dataset['color']=[300]*len(dataset)

dataset['marker_symbol'] = ['x']*len(dataset)

for idx, event in enumerate(dataset.Event):
    if event == 'Visit':
        dataset.marker_symbol.iloc[idx] = 'triangle-up'
    elif event == 'Pass':
        dataset.marker_symbol.iloc[idx] = 'circle'
    elif event == 'Died':
        dataset.marker_symbol.iloc[idx] = 'cross'
    elif event == 'Start' or event == 'End':
        dataset.marker_symbol.iloc[idx] = 'star'
    else: continue
        
dataset['marker_color'] = ['red']*len(dataset)


C:\Users\Kevin\Anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


### Plotly Figure

In [54]:
fig = go.Figure() # Globe Figure

# Trace of route
fig.add_trace(go.Scattergeo(
        name = 'Route',
        lon = dataset.Longitude,
        lat = dataset.Latitude,
        mode = 'markers+lines',
        marker = dict(
        color = dataset.marker_color,
        symbol = dataset.marker_symbol,
        size = 10,
        line=dict(
        color='black',
        width=2)),        
        text = dataset.Name.str.cat(dataset.Date, sep = ', '),
        line = dict(width = 2, color = 'black'
        )))


# Frames
frames1 = [go.Frame(data = [go.Scattergeo(lat=dataset.loc[:k+1, 'Latitude'], 
                                            lon=dataset.loc[:k+1, 'Longitude'])], 
                                            traces=[0],
                                            name=f'frame{k}')for k  in  range(len(dataset))]

fig.update(frames=frames1); 

# Slider
sliders = [dict(steps= [dict(method= 'animate',
                             args=[[f'frame{k}'],
                             dict(mode='immediate',
                             frame=dict(duration=300, redraw= True ),
                             transition=dict(duration = 300, easing = 'cubic-in-out'))],
                             label=str(dataset.timedelta[k])) for k in range(len(dataset))], 
                             transition= dict(duration = 300),
                             x=0,  
                             y=0, 
                             currentvalue=dict(font=dict(size=12), 
                             prefix='Days: ', 
                             visible=True, 
                             xanchor= 'center'),  
                             len=1.0)]

# Make animation buttons
class AnimationButtons():
    def play_scatter(frame_duration = 500, transition_duration = 300):
        return dict(label="Play", method="animate", args=
                    [None, {"frame": {"duration": frame_duration, "redraw": False},
                            "fromcurrent": True, "transition": {"duration": transition_duration, "easing": "cubic-in-out"}}])
    
    def play(frame_duration = 1000, transition_duration = 300):
        return dict(label="Play", method="animate", args=
                    [None, {"frame": {"duration": frame_duration, "redraw": True},
                            "mode":"immediate",
                            "fromcurrent": True, "transition": {"duration": transition_duration, "easing": "cubin-in-out"}}])
    
    def pause():
        return dict(label="Pause", method="animate", args=
                    [[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate", "transition": {"duration": 0}}])

# Globe layout
fig.update_layout(
    title_text = 'Magellan\'s Voyage',
    transition = dict(duration=500,easing='circle-in'),
    font=dict(
    family="Balto, italic",
    size=20,
    color="seagreen"),
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    showlegend = True,
    legend = dict(borderwidth=2),
    sliders = sliders,
    updatemenus=[dict(type="buttons", buttons=[AnimationButtons.play(), AnimationButtons.pause()])],
    geo = dict(
        showland = True,
        showcountries = False,
        showocean = True,
        countrywidth = 0.5,
        landcolor = 'rgb(230, 145, 56)',
        lakecolor = 'rgb(0, 100, 100)',
        oceancolor = 'rgb(0, 100, 100)',
        bgcolor = 'rgba(0,0,0,0)',
        projection = dict(
            type = 'orthographic',
            rotation = dict(
                lon = -100,
                lat = 40,
                roll = 0
            )
        ),
        lonaxis = dict(
            showgrid = True,
            gridcolor = 'rgb(102, 102, 102)',
            gridwidth = 0.5
        ),
        lataxis = dict(
            showgrid = True,
            gridcolor = 'rgb(102, 102, 102)',
            gridwidth = 0.5
        )
    )
)

# For adding static images
fig.add_layout_image(dict(
        source='voyage.png',
        xref= "x",
        yref= "y",
        x= -1,
        y= -2,
        sizex= 5,
        sizey= 6,
        xanchor='left',
        yanchor='bottom',
        sizing= "stretch",
        opacity= 1,
        layer= "below"))

fig.add_layout_image(dict(
        source='voyage2.png',
        xref= "x",
        yref= "y",
        x= 3,
        y= -2,
        sizex= 3,
        sizey= 6,
        xanchor='left',
        yanchor='bottom',
        sizing= "stretch",
        opacity= 1,
        layer= "below"))

# Remove grids and axes labels
fig.update_xaxes(showgrid=False, nticks=0, showline=False, showticklabels=False, zeroline=False)
fig.update_yaxes(showgrid=False, nticks=0, showline=False, showticklabels=False, zeroline=False)

# Show Figure
fig.show()
    

### Dash Attempt

In [3]:
fig = go.Figure() # Globe Figure

# Trace of route
fig.add_trace(go.Scattergeo(
        name = 'Route',
        lon = dataset.Longitude,
        lat = dataset.Latitude,
        mode = 'markers+lines',
        marker = dict(
        color = dataset.marker_color,
        symbol = dataset.marker_symbol,
        size = 10,
        line=dict(
        color='black',
        width=2)),        
        text = dataset.Name,
        line = dict(width = 2, color = 'black'
        )))


# Frames
frames1 = [go.Frame(data = [go.Scattergeo(lat=dataset.loc[:k+1, 'Latitude'], 
                                            lon=dataset.loc[:k+1, 'Longitude'])], 
                                            traces=[0],
                                            name=f'frame{k}')for k  in  range(len(dataset))]

fig.update(frames=frames1); 

# Slider
sliders = [dict(steps= [dict(method= 'animate',
                             args=[[f'frame{k}'],
                             dict(mode='immediate',
                             frame=dict(duration=100, redraw= True ),
                             transition=dict( duration= 0))],
                             label=str(dataset.timedelta[k])) for k in range(len(dataset))], 
                             transition= dict(duration= 0 ),
                             x=0,#slider starting position  
                             y=0, 
                             currentvalue=dict(font=dict(size=12), 
                             prefix='Days: ', 
                             visible=True, 
                             xanchor= 'center'),  
                             len=1.0)]


class AnimationButtons():
    def play_scatter(frame_duration = 500, transition_duration = 300):
        return dict(label="Play", method="animate", args=
                    [None, {"frame": {"duration": frame_duration, "redraw": False},
                            "fromcurrent": True, "transition": {"duration": transition_duration, "easing": "quadratic-in-out"}}])
    
    def play(frame_duration = 1000, transition_duration = 0):
        return dict(label="Play", method="animate", args=
                    [None, {"frame": {"duration": frame_duration, "redraw": True},
                            "mode":"immediate",
                            "fromcurrent": True, "transition": {"duration": transition_duration, "easing": "linear"}}])
    
    def pause():
        return dict(label="Pause", method="animate", args=
                    [[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate", "transition": {"duration": 0}}])

# Globe layout
fig.update_layout(
    title_text = 'Magellan\'s Voyage',
    transition = dict(duration=500,easing='circle-in'),
    font=dict(
    family="Balto, italic",
    size=18,
    color="seagreen"
    ),
    showlegend = True,
    legend = dict(borderwidth=2),
    sliders = sliders,
    updatemenus=[dict(type="buttons", buttons=[AnimationButtons.play(), AnimationButtons.pause()])],
    geo = dict(
        showland = True,
        showcountries = False,
        showocean = True,
        countrywidth = 0.5,
        landcolor = 'rgb(230, 145, 56)',
        lakecolor = 'rgb(0, 100, 100)',
        oceancolor = 'rgb(0, 100, 100)',
        projection = dict(
            type = 'orthographic',
            rotation = dict(
                lon = -100,
                lat = 40,
                roll = 0
            )
        ),
        lonaxis = dict(
            showgrid = True,
            gridcolor = 'rgb(102, 102, 102)',
            gridwidth = 0.5
        ),
        lataxis = dict(
            showgrid = True,
            gridcolor = 'rgb(102, 102, 102)',
            gridwidth = 0.5
        )
    )
)

# For adding static image
fig.add_layout_image(dict(
        source='voyage.png',
        xref= "x",
        yref= "y",
        x= -1,
        y= -2,
        sizex= 5,
        sizey= 6,
        xanchor='left',
        yanchor='bottom',
        sizing= "stretch",
        opacity= 1,
        layer= "below"))

fig.add_layout_image(dict(
        source='voyage2.png',
        xref= "x",
        yref= "y",
        x= 3,
        y= -2,
        sizex= 3,
        sizey= 6,
        xanchor='left',
        yanchor='bottom',
        sizing= "stretch",
        opacity= 1,
        layer= "below"))

fig.update_xaxes(showgrid=False, nticks=0, showline=False, showticklabels=False, zeroline=False)
fig.update_yaxes(showgrid=False, nticks=0, showline=False, showticklabels=False, zeroline=False)

# Tried to make a dash app with images appearing on hover but dash is awful to work with, I tried for two days to put an 
# image in the background and it doesn't want to work so I gave up.

fig.update_traces(hoverinfo="none", hovertemplate=None)

app = Dash(__name__)

app.layout = html.Div([
    html.Div([
        html.Div([
            dcc.Graph(id='graph-basic-2', figure=fig, clear_on_unhover=True),
            dcc.Tooltip(id='graph-tooltip', direction='bottom')], 
            className="container")])])

@app.callback(
    Output("graph-tooltip", "show"),
    Output("graph-tooltip", "bbox"),
    Output("graph-tooltip", "children"),
    Input("graph-basic-2", "hoverData"),
)

def display_hover(hoverData):
    if hoverData is None:
        return False, no_update, no_update

    pt = hoverData["points"][0]
    bbox = pt["bbox"]
    num = pt["pointNumber"]

    dataset_row = dataset.iloc[num]
    img_src = dataset_row['IMG_URL']
    if type(dataset_row['TEXT']) == str:
        text = dataset_row['TEXT']
    else:
        text = str(dataset_row['Latitude'])+ ' , ' + str(dataset_row['Longitude'])

    children = [
        html.Div([
            html.Img(src=img_src, style={"width": "100%"}),
            html.P(f"{text}"),],
            style={'width': '200px', 'white-space': 'normal'})]

    return True, bbox, children


if __name__ == "__main__":
    app.run_server(debug=True, use_reloader=False)
    

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
